In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("./data/train_dataset.csv")

data['is_train'] = True
evaluation = pd.read_csv("./data/evaluation_public.csv")
evaluation['is_train'] = False
sample = pd.read_csv("./data/sample_submission.csv")

data = pd.concat([data, evaluation]).reset_index(drop=True)

# 特征处理

In [3]:
data['date'] = pd.to_datetime(data['time'])
data['hour'] = data['date'].dt.hour
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['minute'] = data['date'].dt.minute
data['weekday'] = data['date'].dt.weekday
data['day'] = data['date'].dt.day
data['hour'] = data['date'].dt.hour
# data['ts'] = data['hour']*3600 + data['minute']*60 + data['date'].dt.second
data['ts'] = data['hour']*60 + data['minute']


In [4]:
import gc
features = [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'CS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP','weekday','hour', 'ts']

filter_set = {'time', 'Label1', 'Label2','CS_LL','CS_NH3', 'JS_SW', 'B_QY_ORP'}
features = [f for f in features if f not in filter_set] #
for f in features:
    data[f] = data[f].fillna(method='ffill')

# 提取特征的diff值

In [5]:
df_list = []
i = 0
add_featuers = []
length = 0
# JS_TN, MCCS_NO3,MCCS_NH4,'B_QY_ORP'
filter_set = {'weekday', 'hour', 'ts', 'B_QY_ORP', 'JS_TN', 'CS_SW', 'MCCS_NH4', 'N_HYC_JS_DO', 'MCCS_NO3', 'JS_SW'}
# filter_set = {'weekday', 'hour', 'ts', 'MCCS_NO3'}
for f in features:
    if f in filter_set: continue
    for r in [15]:
        train_rolling = data[f].rolling(window=r, center=False)
        f_mean_name = 'rolling{}_{}_mean'.format(r,f)
        f_max_name = 'rolling{}_{}_max'.format(r,f)
        f_min_name = 'rolling{}_{}_min'.format(r,f)
        f_std_name = 'rolling{}_{}_std'.format(r,f)
        f_corr_name = 'rolling{}_{}_corr'.format(r,f)
        f_cov_name = 'rolling{}_{}_cov'.format(r,f)
        f_skew_name = 'rolling{}_{}_skew'.format(r,f)
        f_kurt_name = 'rolling{}_{}_kurt'.format(r,f)
        data[f_mean_name] = train_rolling.mean().fillna(0).values
        data[f_max_name] = train_rolling.max().fillna(0).values
        data[f_min_name] = train_rolling.min().fillna(0).values
        data[f_std_name] = train_rolling.std().fillna(0).values
        data[f_corr_name] = train_rolling.corr().fillna(0).values
        data[f_skew_name] = train_rolling.skew().fillna(0).values

        data[f'{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
        data[f'{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
        data[f'{f}_{f_min_name}_cha'] = data[f] - data[f_min_name]

        data[f'{f}_{f_min_name}_radio'] = data[f'{f}_{f_mean_name}_cha']/data[f]
        data[f'{f}_{f_max_name}_radio'] = data[f'{f}_{f_max_name}_cha']/data[f]
        data[f'{f}_{f_mean_name}_radio'] = data[f'{f}_{f_min_name}_cha']/data[f]
        data[f'{f}_diff'] = data[f].diff()

        if i == 0:
            add_featuers.append(f_mean_name)
            add_featuers.append(f_max_name)
            add_featuers.append(f_min_name)
            add_featuers.append(f_std_name)
            add_featuers.append(f_corr_name)
            add_featuers.append(f_skew_name)
            add_featuers.append(f'{f}_{f_mean_name}_cha')
            add_featuers.append(f'{f}_{f_max_name}_cha')
            add_featuers.append(f'{f}_{f_min_name}_cha')
            add_featuers.append(f'{f}_diff')

features.extend(add_featuers)


gc.collect()
# 对所有的特征进行划分
for f in features:
    if f not in ['weekday','hour', 'ts']:
        q = len(data[f].drop_duplicates())

        data[f] = pd.qcut(data[f], q=int(q/10), labels=False, duplicates="drop")

all_train = data[data['is_train']].reset_index(drop=True)
test = data[~data['is_train']].reset_index(drop=True)

C:\Users\38125\AppData\Local\Temp/ipykernel_65484/1814523910.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{f}_{f_mean_name}_radio'] = data[f'{f}_{f_min_name}_cha']/data[f]
C:\Users\38125\AppData\Local\Temp/ipykernel_65484/1814523910.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{f}_diff'] = data[f].diff()
C:\Users\38125\AppData\Local\Temp/ipykernel_65484/1814523910.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has 

# 删除分布不均衡的特征

In [6]:
all_train = all_train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
def transform(x: pd.Series, c=20):
    return np.log1p(x/c)

def inverse_transform(x: pd.Series, c = 20):
    return np.expm1(x)*c

all_train = all_train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
all_train['Label1'] = transform(all_train['Label1'])
label_c = 8
all_train['Label2'] = transform(all_train['Label2'], c=label_c)

In [7]:
import matplotlib.pyplot as plt
def plot(y1, y2, x_title, y_title, title):
    plt.figure(figsize=(16,8))
    plt.plot(range(len(y1)), y1)
    plt.plot(range(len(y1)), y2)
    plt.legend()
    plt.xlabel(x_title)
    plt.ylabel(y_title)
    plt.title(title)
    plt.show()

In [29]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
score_list = []
labels = ['Label1']
test_size = 4000

for label in labels:
    X_train = all_train[features][:-test_size]
    y_train = all_train[label][:-test_size]

    X_test = all_train[features][-test_size:]
    y_test = all_train[label][-test_size:]
    if label == "Label1":
        model = xgb.XGBRegressor(
            max_bin=150,
            boosting="gbdt",
            max_depth=6,
            learning_rate=0.02,
            n_estimators=10000,
            subsample = 0.6,
            colsample_bytree=0.3,
            min_child_weight=0.01,
            bagging_seed=1,
            reg_alpha=1,
            reg_lambda=1,  # 此处不改了
            gpu_id=0,
            tree_method='gpu_hist',
            random_state=10215
        )
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['rmse'],
              early_stopping_rounds=150, verbose=200)

        test_pred = model.predict(X_test)
        score_list.append(np.sqrt(mean_squared_error(inverse_transform(y_test), inverse_transform(test_pred))))
        print(f"score_list = {score_list}")
        # model2 = xgb.XGBRegressor(
        #     max_bin=50,
        #     boosting="gbdt",
        #     max_depth=4,
        #     learning_rate=0.03,
        #     n_estimators=int(1.2*model.best_iteration),
        #     subsample = 0.4,
        #     colsample_bytree=0.5,
        #     min_child_weight=1,
        #     bagging_seed=1,
        #     reg_alpha=1,
        #     reg_lambda=1,  # 此处不改了
        #     gpu_id=0,
        #     tree_method='gpu_hist',
        #     random_state=1212
        # )
        # model2.fit(all_train[features], all_train[label])
        # test[label] = inverse_transform(model2.predict(test[features]))
    else:
        model = xgb.XGBRegressor(
            max_bin=40,
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.03,
            n_estimators=10000,
            subsample = 0.4,
            colsample_bytree=0.4,
            min_child_weight=0.01,
            bagging_seed=1,
            reg_alpha=1,
            reg_lambda=1.5,  # 此处不改了
            gpu_id=0,
            tree_method='gpu_hist',
            random_state=1212
        )

        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['rmse'],
              early_stopping_rounds=150, verbose=200)
        test_pred = model.predict(X_test)
        score_list.append(np.sqrt(mean_squared_error(inverse_transform(y_test, c=label_c), inverse_transform(test_pred, c=label_c))))
        print(f"score_list = {score_list}")
        # model2 = xgb.XGBRegressor(
        #     max_bin=40,
        #     boosting="gbdt",
        #     max_depth=4,
        #     learning_rate=0.03,
        #     n_estimators=int(1.2*model.best_iteration),
        #     subsample = 0.4,
        #     colsample_bytree=0.4,
        #     min_child_weight=0.01,
        #     bagging_seed=1,
        #     reg_alpha=1,
        #     reg_lambda=1.5,  # 此处不改了
        #     gpu_id=0,
        #     tree_method='gpu_hist',
        #     random_state=1212
        # )
        # model2.fit(all_train[features], all_train[label])
        #
        # test[label] = inverse_transform(model2.predict(test[features]), c=label_c)
loss = np.mean(score_list)
score = 1000/(1+loss)
print(score)

[2000]	validation_0-rmse:0.04270	validation_1-rmse:0.10016
[2198]	validation_0-rmse:0.04112	validation_1-rmse:0.10013
score_list = [1304.5049828049457]
0.7659871185259268
[11:31:01] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "bagging_seed", "boosting" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:5.88332	validation_1-rmse:5.82457


c:\users\38125\anaconda3\envs\python38\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\users\38125\anaconda3\envs\python38\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[200]	validation_0-rmse:0.14520	validation_1-rmse:0.14912
[400]	validation_0-rmse:0.07744	validation_1-rmse:0.10727
[600]	validation_0-rmse:0.06609	validation_1-rmse:0.10514
[800]	validation_0-rmse:0.05907	validation_1-rmse:0.10384
[1000]	validation_0-rmse:0.05421	validation_1-rmse:0.10300
[1200]	validation_0-rmse:0.05061	validation_1-rmse:0.10249
[1400]	validation_0-rmse:0.04775	validation_1-rmse:0.10232
[1600]	validation_0-rmse:0.04536	validation_1-rmse:0.10208
[1800]	validation_0-rmse:0.04332	validation_1-rmse:0.10170
[2000]	validation_0-rmse:0.04154	validation_1-rmse:0.10166
[2200]	validation_0-rmse:0.03996	validation_1-rmse:0.10159
[2400]	validation_0-rmse:0.03857	validation_1-rmse:0.10155
[2600]	validation_0-rmse:0.03731	validation_1-rmse:0.10148
[2796]	validation_0-rmse:0.03622	validation_1-rmse:0.10144
score_list = [1325.9383228999868]
0.7536145295845611


In [9]:
loss = np.mean(score_list)
score = 1000/(1+loss)
print(score)

0.8005763573191209


In [10]:
test[['time'] + labels].to_csv(f"./res/lightgbm_random_res.csv", index=False)

In [11]:
loss = np.mean(score_list)
score = 1000/(1+loss)
print(score)

0.8005763573191209
